In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math

In [ ]:
# def calculate_angle(a, b, c):
#     radians = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
#     angle = math.degrees(radians)
#     angle = angle % 360
#     if angle > 180:
#         angle = 360 - angle
#     return angle
def calculate_angle(a, b, c):
    vector_ab = (b[0] - a[0], b[1] - a[1], b[2] - a[2])
    vector_cb = (b[0] - c[0], b[1] - c[1], b[2] - c[2])

    # Calculate the dot product
    dot_product = vector_ab[0] * vector_cb[0] + vector_ab[1] * vector_cb[1] + vector_ab[2] * vector_cb[2]

    # Calculate the magnitudes of the vectors
    magnitude_ab = math.sqrt(vector_ab[0]**2 + vector_ab[1]**2 + vector_ab[2]**2)
    magnitude_cb = math.sqrt(vector_cb[0]**2 + vector_cb[1]**2 + vector_cb[2]**2)

    # Calculate the cosine of the angle
    cosine_angle = dot_product / (magnitude_ab * magnitude_cb)

    # Convert cosine to angle in degrees
    angle = math.degrees(math.acos(cosine_angle))
    return angle

In [ ]:
def plot_world_landmarks(
    plt,
    ax,
    landmarks,
    visibility_th=0.5,
):
    landmark_point = []

    for index, landmark in enumerate(landmarks.landmark):
        landmark_point.append(
            [landmark.visibility, (landmark.x, landmark.y, landmark.z)])

    face_index_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    right_arm_index_list = [11, 13, 15, 17, 19, 21]
    left_arm_index_list = [12, 14, 16, 18, 20, 22]
    right_body_side_index_list = [11, 23, 25, 27, 29, 31]
    left_body_side_index_list = [12, 24, 26, 28, 30, 32]
    shoulder_index_list = [11, 12]
    waist_index_list = [23, 24]

    face_x, face_y, face_z = [], [], []
    for index in face_index_list:
        point = landmark_point[index][1]
        face_x.append(point[0])
        face_y.append(point[2])
        face_z.append(point[1] * (-1))

    right_arm_x, right_arm_y, right_arm_z = [], [], []
    for index in right_arm_index_list:
        point = landmark_point[index][1]
        right_arm_x.append(point[0])
        right_arm_y.append(point[2])
        right_arm_z.append(point[1] * (-1))

    left_arm_x, left_arm_y, left_arm_z = [], [], []
    for index in left_arm_index_list:
        point = landmark_point[index][1]
        left_arm_x.append(point[0])
        left_arm_y.append(point[2])
        left_arm_z.append(point[1] * (-1))

    right_body_side_x, right_body_side_y, right_body_side_z = [], [], []
    for index in right_body_side_index_list:
        point = landmark_point[index][1]
        right_body_side_x.append(point[0])
        right_body_side_y.append(point[2])
        right_body_side_z.append(point[1] * (-1))

    left_body_side_x, left_body_side_y, left_body_side_z = [], [], []
    for index in left_body_side_index_list:
        point = landmark_point[index][1]
        left_body_side_x.append(point[0])
        left_body_side_y.append(point[2])
        left_body_side_z.append(point[1] * (-1))

    shoulder_x, shoulder_y, shoulder_z = [], [], []
    for index in shoulder_index_list:
        point = landmark_point[index][1]
        shoulder_x.append(point[0])
        shoulder_y.append(point[2])
        shoulder_z.append(point[1] * (-1))

    waist_x, waist_y, waist_z = [], [], []
    for index in waist_index_list:
        point = landmark_point[index][1]
        waist_x.append(point[0])
        waist_y.append(point[2])
        waist_z.append(point[1] * (-1))

    ax.cla()
    ax.set_xlim3d(-1, 1)
    ax.set_ylim3d(-1, 1)
    ax.set_zlim3d(-1, 1)

    ax.scatter(face_x, face_y, face_z)
    ax.plot(right_arm_x, right_arm_y, right_arm_z)
    ax.plot(left_arm_x, left_arm_y, left_arm_z)
    ax.plot(right_body_side_x, right_body_side_y, right_body_side_z)
    ax.plot(left_body_side_x, left_body_side_y, left_body_side_z)
    ax.plot(shoulder_x, shoulder_y, shoulder_z)
    ax.plot(waist_x, waist_y, waist_z)

    plt.pause(.001)

    return

In [ ]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
cap1 = cv2.VideoCapture('../videos/dancer.mp4')
cap2 = cv2.VideoCapture('../videos/right_dance.mp4')

# skips 'cap2' frames
# skips = 492
# for _ in range(skips):
#     ret, frame = cap2.read()

pose1 = mp_pose.Pose(model_complexity=1)
pose2 = mp_pose.Pose(model_complexity=1)

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    if not (ret1 and ret2):
        break

    results1 = pose1.process(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
    results2 = pose2.process(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))

    # blank the frames
    height1, width1, _ = frame1.shape
    height2, width2, _ = frame2.shape
    frame1 = np.zeros((height1, width1, 3), dtype=np.uint8)
    frame2 = np.zeros((height2, width2, 3), dtype=np.uint8)

    mp_drawing.draw_landmarks(
            frame1,
            results1.pose_landmarks,
            mp.solutions.pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    mp_drawing.draw_landmarks(
            frame2,
            results2.pose_landmarks,
            mp.solutions.pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


    if results1.pose_landmarks and results2.pose_landmarks:
        landmarks1 = results1.pose_landmarks.landmark
        landmarks1_3d = results1.pose_world_landmarks.landmark
        landmarks2 = results2.pose_landmarks.landmark
        landmarks2_3d = results2.pose_world_landmarks.landmark

        mark = mp_pose.PoseLandmark

        # Draw circle on elbow
        frame1_height, frame1_width, _ = frame1.shape
        left_elbow1_x = int(landmarks1[mark.LEFT_ELBOW].x * frame1_width)
        left_elbow1_y = int(landmarks1[mark.LEFT_ELBOW].y * frame1_height)
        cv2.circle(frame1, (left_elbow1_x, left_elbow1_y), 5, (255, 0, 0), -1)

        frame2_height, frame2_width, _ = frame2.shape
        left_elbow2_x = int(landmarks2[mark.LEFT_ELBOW].x * frame2_width)
        left_elbow2_y = int(landmarks2[mark.LEFT_ELBOW].y * frame2_height)
        cv2.circle(frame2, (left_elbow2_x, left_elbow2_y), 5, (255, 0, 0), -1)

        # Calculate angles
        angle_elbow_1 = calculate_angle(
            (landmarks1_3d[mark.LEFT_SHOULDER].x, landmarks1_3d[mark.LEFT_SHOULDER].y, landmarks1_3d[mark.LEFT_SHOULDER].z),
            (landmarks1_3d[mark.LEFT_ELBOW].x, landmarks1_3d[mark.LEFT_ELBOW].y, landmarks1_3d[mark.LEFT_ELBOW].z),
            (landmarks1_3d[mark.LEFT_WRIST].x, landmarks1_3d[mark.LEFT_WRIST].y, landmarks1_3d[mark.LEFT_WRIST].z)
        )

        angle_elbow_2 = calculate_angle(
            (landmarks2_3d[mark.LEFT_SHOULDER].x, landmarks2_3d[mark.LEFT_SHOULDER].y, landmarks2_3d[mark.LEFT_SHOULDER].z),
            (landmarks2_3d[mark.LEFT_ELBOW].x, landmarks2_3d[mark.LEFT_ELBOW].y, landmarks2_3d[mark.LEFT_ELBOW].z),
            (landmarks2_3d[mark.LEFT_WRIST].x, landmarks2_3d[mark.LEFT_WRIST].y, landmarks2_3d[mark.LEFT_WRIST].z)
        )

        # Display angles on frames
        cv2.putText(frame1, f"{angle_elbow_1}", (left_elbow1_x, left_elbow1_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame2, f"{angle_elbow_2}", (left_elbow2_x, left_elbow2_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('frame1', frame1)
    cv2.imshow('frame2', frame2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

In [ ]:
from numpy.linalg import norm
from numpy import dot

def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))